In [1]:
################ convert pdf to images ########################

from pdf2image import convert_from_path
import os

images_path = os.getcwd()

def pdf_to_images(pdf, filename):
    pdf = pdf.replace('\\', '\\\\')
    # print(pdf)
    try:
        images = convert_from_path(pdf)
        for i in range(len(images)):
            images[i].save(images_path+'\\images3\\{}_page{}'.format(filename, i) +'.jpg', 'JPEG')
    except:
        pass

pdf_path = 'C:\\Users\\Lenovo\\Downloads\\AMR Billing Oct to Dec 2023'

#store all the file names in this list
filelist = []

for root, dirs, files in os.walk(pdf_path):
	for file in files:
        #append the file name to the list
		filelist.append(os.path.join(root,file))


# print all the file names
for file in filelist:
    filename = os.path.split(file)[-1].split('.pdf')[0]
    if ' ' in filename:
        filename = filename.replace(' ', '_')
    # print(filename)
    try:
        pdf_to_images(file, filename)
    except:
        pass

In [4]:
print(os.getcwd())

c:\air_ticket\netlink_electricity


In [2]:
########################## parse output of preprocessed images #############################

from PIL import Image, ImageEnhance
import cv2
import numpy as np
from skimage import io
from skimage.transform import rotate
from skimage.color import rgb2gray
from deskew import determine_skew
from matplotlib import pyplot as plt
from pdf2image import convert_from_path
import shutil
import glob
import os

def set_dpi(image, preproc_image):
    im = Image.open(image)
    im.save(preproc_image, dpi=(600,600))
    return preproc_image

def increase_contrast(image, preproc_image):
    im = Image.open(image)
    enhancer = ImageEnhance.Contrast(im)
    factor = 2 #increase contrast
    im_output = enhancer.enhance(factor)
    im_output.save(preproc_image)
    return preproc_image

def increase_sharpness(image, preproc_image):
    im = Image.open(image)
    enhancer = ImageEnhance.Sharpness(im)
    factor = 2
    im_output = enhancer.enhance(factor)
    im_output.save(preproc_image)
    return preproc_image

def convert_to_grayscale(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def convert_to_rgb(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def otsu_threshold(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    # im = cv2.threshold(im, 120, 255, cv2.THRESH_BINARY, cv2.THRESH_OTSU)[1]
    im = cv2.adaptiveThreshold(im,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def remove_noise(image, preproc_image):
    im = cv2.imread(image)
    im = cv2.fastNlMeansDenoising(im, None, 20, 7, 21)
    cv2.imwrite(preproc_image, im)
    return preproc_image

def deskew(image):
    im = io.imread(image)
    angle = determine_skew(im)
    rotated = rotate(im, angle, resize=True) * 255
    io.imsave(image, rotated.astype(np.uint8))
    return image

combinations = ['convert_to_rgb-set_dpi']
# combinations = ['set_dpi-increase_contrast-increase_sharpness']

image_path = os.getcwd()
for image in glob.glob(image_path+'\\images3\\*.jpg'):
    # filename = os.path.split(image)[-1].split('.jpg')[0]
    # print(filename)
    # preproc_image = 'preprocessed_ensemble\\'+filename+'.jpg'
    # shutil.copy(image, preproc_image)
    for combination in combinations:
        filename = os.path.split(image)[-1].split('.jpg')[0]
        preproc_image = image_path+'\\preprocessed_images3\\'+filename+'.jpg'
        shutil.copy(image, preproc_image)
        methods = combination.split('-')
        for method in methods:
            if 'deskew' in method:
                preproc_image = eval(method+'(preproc_image)')
            else:
                preproc_image = eval(method+'(preproc_image, preproc_image)')
        # print(combination)
        image_name = image_path+'\\preprocessed_images3\\'+filename+'_'+combination+'.jpg'
        img = cv2.imread(preproc_image)
        cv2.imwrite(image_name, img)

        psm_values = [6]

        for psm in psm_values:

            parsed_output = image_path+'\\output3\\'+filename+'_'+combination+'_'+str(psm)
            command = 'tesseract {} {} -l eng --psm {}'.format(preproc_image, parsed_output, psm)
            os.system(command)

In [3]:
################### ensemble the output files #####################

import glob
import os

def ensemble_output():

    for file in glob.glob('C:/air_ticket/appcino_all_docs/output3/*.txt'):
        filename = os.path.split(file)[-1].split('.txt')[0]
        ind = filename.index('page')
        filename = filename[:ind-1]
        # print(filename)
        with open(file, encoding="utf-8") as fp:
                # read the file
                text = fp.read()
                # open new file and append text of files for the same name 
                with open('C:/air_ticket/appcino_all_docs/ensemble_files3/{}.txt'.format(filename), 'a') as file:
                    # to separate the text of multiple configuration using ---
                    file.write(text+"\n---"+"\n")

ensemble_output()

In [8]:
import glob
import datetime
import os
import pandas as pd

def get_traveler_name(text_pages):
    all_names = []
    names = []
    for i in range(len(text_pages)):
        # print('hello')
        page = text_pages[i].splitlines()
        page = [line for line in page if not line.isspace() and len(line) > 0]
        for j in range(len(page)):
            if 'air ticket' in page[j].lower() and 'for' not in page[j].lower() and 'preferred seat' not in page[j+1].lower():
                if 'mr.' not in page[j].lower():
                    for k in range(j+1, len(page)):
                        if 'sector' in page[k].lower():
                            # print('names---> ', names)
                            all_names.append(names)
                            names = []
                            break
                        else:
                            names.append(page[k])
                else:
                    name = ''
                    for k in range(j, len(page)):
                        if 'sector' in page[k].lower():
                            # print('names---> ', names)
                            all_names.append(names)
                            names = []
                            break
                        lower_string = page[k].lower()
                        # print('lower_string---> ', lower_string)
                        # print('lower string words---> ', lower_string.split())
                        ind = lower_string.split().index('mr.')
                        # print('ind---> ', ind)
                        words = page[k].split()[ind:]
                        # print(words)
                        for word in words:
                            if not word.strip('.').isalpha():
                                # print('name---> ', name)
                                break
                            name += word + ' '
                        # # print(name)
                        if name:
                            names.append(name.strip())
                            name = ''
                        # else:
                        #     names.append(page[k])
            
            if 'balance due' in page[j].lower() or 'terms & conditions' in page[j].lower():
                all_names = clean_traveler_name(all_names)
                return all_names

            
    # [['Mr. Siddharth Goyal', 'Mr. Ajay Sharma'], ['Preferred Seat', 'Selection Charges Inc.', 'Sc'], ['Mr. Siddharth Goyal', 'Mr. Ajay Sharma']]
            
def clean_traveler_name(all_names):
    debarred = ['preferred', 'selection', 'air', 'chares', 'sc', 'booking']
    
    for x in all_names.copy():
        for y in x.copy():
            for d in debarred:
                if d in y.lower():
                    x.remove(y)
                    break

    all_names = [sublist for sublist in all_names if sublist]

    for i in range(len(all_names.copy())):
        for j in range(len(all_names[i].copy())):
            if 'mr.' not in all_names[i][j].lower() and 'ms.' not in all_names[i][j].lower():
                all_names[i][j-1] = all_names[i][j-1]+' '+all_names[i][j]
                all_names[i].remove(all_names[i][j])

    return all_names

def get_start_date(text_pages):
    start_date = ''
    start_dates = []
    for i in range(len(text_pages)):
        page = text_pages[i].splitlines()
        page = [line for line in page if not line.isspace() and len(line) > 0]
        for j in range(len(page)):
            if 'air ticket' in page[j].lower() and 'for' not in page[j].lower() and 'preferred seat' not in page[j+1].lower():
                start_date = ''
                flag = 0
                for k in range(j+1, len(page)):
                    if 'travel date' in page[k].lower() and not 'old travel date' in page[k].lower():
                        for t in range(k, len(page)):
                            if 'airline' in page[t].lower() or 'return' in page[t].lower() or 'for' in page[t].lower():
                                start_dates.append(start_date.strip())
                                start_date = ''
                                flag = 1
                                break
                            if 'date' in page[t].lower():
                                start_date += page[t].split(':')[-1]+' '
                                # print('start date---> ', start_date)
                            else:
                                start_date += page[t] + ' '
                    if flag:
                        break
            
            if 'balance due' in page[j].lower() or 'terms & conditions' in page[j].lower():
                start_dates = clean_date(start_dates)
                return start_dates
            
def get_end_date(dates):
    end_date = ''
    end_dates = []
    flag = 0
    for i in range(len(text_pages)):
        page = text_pages[i].splitlines()
        page = [line for line in page if not line.isspace() and len(line) > 0]
        for j in range(len(page)):
            if 'air ticket' in page[j].lower() and 'for' not in page[j].lower() and 'preferred seat' not in page[j+1].lower():
                end_date = ''
                flag = 0
                for k in range(j+1, len(page)):
                    if 'air ticket' in page[k].lower():
                        break
                    if 'return' in page[k].lower():
                        for t in range(k, len(page)):
                            if 'airline' in page[t].lower() or 'for' in page[t].lower():
                                end_dates.append(end_date.strip())
                                end_date = ''
                                flag = 1
                                break
                            if 'return' in page[t].lower():
                                end_date += page[t].split(':')[-1]+' '
                                # print('start date---> ', start_date)
                            else:
                                end_date += page[t] + ' '
                    if flag:
                        break

            
            if 'balance due' in page[j].lower() or 'terms & conditions' in page[j].lower():
                end_dates = clean_date(end_dates)
                return end_dates
            
def clean_date(dates):
    extra_ch = ['*', '’', ' ', '\'']
    for i in range(len(dates)):
        for ec in extra_ch:
            if ec in dates[i]:
                dates[i] = dates[i].replace(ec, '')

    for i in range(len(dates)):
        if '-' not in dates[i]:
            dates[i] = dates[i][:2]+' '+dates[i][2:5]+' '+dates[i][5:]

    formats = ['%d %b %Y', '%d %b %y', '%d-%m-%Y']
    for i in range(len(dates)):
        for format in formats:
            try:
                dates[i] = datetime.datetime.strptime(dates[i], format).strftime('%Y-%m-%d')
            except:
                pass

    return dates

def get_sectors(text_pages):
    sectors = []
    sector = ''
    for i in range(len(text_pages)):
        page = text_pages[i].splitlines()
        page = [line for line in page if not line.isspace() and len(line) > 0]
        for j in range(len(page)):
            if 'air ticket' in page[j].lower() and 'for' not in page[j].lower():
                flag = 0
                for k in range(j+1, len(page)):
                    if 'sector' in page[k].lower():
                        # print('hellooooo--> ', sector)
                        for l in range(k, len(page)):
                            if 'travel' in page[l].lower():
                                sectors.append(sector.strip())
                                sector = ''
                                flag = 1
                                break
                            sector += page[l].split(':')[-1] + ' '
                    if flag:
                        break
            # if flag:
                # break
        
            if 'balance due' in page[j].lower() or 'terms & conditions' in page[j].lower():
                return sectors

def divide_sectors(sectors):
    src = []
    dest = []
    for sector in sectors:
        if '-' in sector:
            l = sector.split("-")
        else:
            l = sector.split()
        s = []
        d = []
        for i in range(len(l)-1):
            s.append(l[i].strip())
            d.append(l[i+1].strip())
        src.append(s)
        dest.append(d)
    
    return src, dest

def get_city_names(sectors):
    cities = []
    countries = []
    
    for sector in sectors:
        try:
            if len(sector) > 3:
                # print('sector---> ', sector)
                cities.append(sector)
                countries.append('None')
                continue
            # print(code)
            value = data[data['iata'] == sector.strip().upper()].iloc[0]
            # print(value)
            cities.append(value['city'])
            countries.append(value['country'])
        except Exception as e:
            cities.append('None') 
            countries.append('None')
    
    return cities, countries
            
def get_final_start_dates(start_dates, traveler_names):
    final_start_dates = []
    for i in range(len(traveler_names)):
        if start_dates:
            final_start_dates.extend([start_dates[i]]*len(traveler_names[i]))
        else:
            final_start_dates.extend(['None']*len(traveler_names[i]))
    return final_start_dates
                        
all_traveler_names = []
all_start_dates = []
all_end_dates = []
all_sectors = []
all_filenames = []
all_source_cities = []
all_dest_cities = []

for file in glob.glob('C:\\air_ticket\\appcino_all_docs\\ensemble_files\\*'):
    print(file.split('\\')[-1])
    filename = os.path.split(file)[-1]
    data = pd.read_excel('World_DB.xlsx')
    
    with open(file) as fp:
        text = fp.read()
        text_pages = text.split('---')[:-1]

        traveler_names = get_traveler_name(text_pages)
        new_traveler_names = [item for sublist in traveler_names for item in sublist]

        start_dates = get_start_date(text_pages)
        start_dates.extend(['None']*(len(traveler_names)-len(start_dates)))
        final_start_dates = get_final_start_dates(start_dates, traveler_names)

        end_dates = get_end_date(text_pages)
        end_dates.extend(['None']*(len(start_dates)-len(end_dates)))
        final_end_dates = get_final_start_dates(end_dates, traveler_names)

        sectors = get_sectors(text_pages)
        sectors.extend(['None']*(len(final_start_dates)-len(sectors)))

        source_cities = divide_sectors(sectors)[0]
        dest_cities = divide_sectors(sectors)[-1]

        # print('names---> ', len(new_traveler_names))
        # # print('start_dates---> ', len(start_dates))
        # print('final start dates---> ', len(final_start_dates))
        # print('final end dates---> ', len(final_end_dates))
        # print('sectors---> ', len(sectors))
        # print('source cities---> ', len(source_cities))
        # print('dest cities---> ', len(dest_cities))

        # print('names---> ', new_traveler_names)
        # # print('start_dates---> ', start_dates)
        # print('final start dates---> ', final_start_dates)
        # print('final end dates---> ', final_end_dates)
        # print('sectors---> ', sectors)
        # print('source cities---> ', source_cities)
        # print('dest cities---> ', dest_cities)

        # print()

        tnames_total = []
        for i in range(len(new_traveler_names)):
            l = len(source_cities[i])
            fn = filename.split('.txt')[0]
            all_filenames.extend([fn]*l)
            all_traveler_names.extend([new_traveler_names[i]]*l)
            tnames_total = [new_traveler_names[i]]*l
            # invoice_sheet.extend(invoice_final*len(tnames_total))
            all_source_cities.extend(source_cities[i])
            all_dest_cities.extend(dest_cities[i])
            all_start_dates.extend([final_start_dates[i]]*l)
            all_end_dates.extend([final_end_dates[i]]*l)
            # invoice_date_sheet.extend([invoice_date_final]*l)

        # all_traveler_names.extend(new_traveler_names)
        # all_start_dates.extend(final_start_dates)
        # all_end_dates.extend(final_end_dates)
        # all_sectors.extend(sectors)
        


        # if len(traveler_names) != len(start_dates):
        #     print(traveler_names)
        #     print(start_dates)

f_source_cities = get_city_names(all_source_cities)[0]
f_dest_cities = get_city_names(all_dest_cities)[0]

source_cont = get_city_names(all_source_cities)[1]
dest_cont = get_city_names(all_dest_cities)[1]

print(len(all_filenames))
print(len(all_traveler_names))
print(len(f_source_cities))
print(len(f_dest_cities))
print(len(source_cont))
print(len(dest_cont))
print(len(all_start_dates))
print(len(all_end_dates))

#352-2023_Appcino_Tech._Pvt_Ltd.txt
#354-2023_Appcino_Technologies.txt
#355-2023_Appcino_Technologies_Pvt_Ltd.txt
#358-2023.txt


KeyboardInterrupt: 

In [47]:
all_names = [['Mr. Siddharth Goyal', 'Mr. Ajay Sharma'], ['Preferred Seat', 'Selection Charges Inc.', 'Sc'], ['Mr. Siddharth Goyal', 'Mr. Ajay Sharma']]

debarred = ['preferred', 'selection', 'sc']
for x in all_names.copy():
    for y in x.copy():
        for d in debarred:
            if d in y.lower():
                x.remove(y)
                break

all_names = [sublist for sublist in all_names if sublist]

print(all_names)

[['Mr. Siddharth Goyal', 'Mr. Ajay Sharma'], ['Mr. Siddharth Goyal', 'Mr. Ajay Sharma']]


In [8]:
x = 'Sector: MUM-HYD'
x.split(':')

['Sector', ' MUM-HYD']

In [62]:
import pandas as pd

df = pd.DataFrame({
    'Filename': all_filenames,
    'Traveler Name': all_traveler_names,
    'Source City': f_source_cities,
    'Source Country': source_cont,
    'Destination City': f_dest_cities,
    'Destination Country': dest_cont,
    'Start Date': all_start_dates,
    'End Date': all_end_dates
})

df.to_excel('appcino_airtravel.xlsx')